# T2DM islet scRNA-seq dataset generation pipeline

This notebook describes the following pipeline:
<ol>
<li><b>Retrieval of FASTQ files</b></li>
<li><b>Processing of the FASTQ files</b></li>
<li><b>Mapping of the FASTQ files</b></li>
<li><b>Counting of genes</b></li>
<li><b>Quality control and filtering</b></li>
<li><b>Gene expression analysis</b></li>
</ol>

## Download fastq files
Raw sequencing fastq files were downloaded in several ways depending on the database where they are located.
<ol>
<li>Sequence Read Archive (SRA) data from NCBI can be retreaved via the <b>SRA toolkit</b></li>
<li>ArrayExpress Archive of Functional Genomics Data from EMBL-EBI can be retreaved via <b>ArrayExpress</b></li>   
</ol>

#### Requiered software
<ul><li>Bash</li>
<li>SRA toolkit</li>
</ul>

>### SRA toolkit
First go to the GEO website (https://www.ncbi.nlm.nih.gov/geo/) and search for the dataset with the GSE number. Next scroll to the bottom of the page and go to the SRA Run Selector website, here the <b>Accesion list</b> (SRR_Acc_list.txt) can be downloaded.  
>
><code>prefetch -X <b>100G</b> -p --option-file <b>SRR_Acc_list.txt</b></code>
>
><code>for srr in &#0036;(awk '{print &#0036;0}' SRR_Acc_list.txt); 
</code>><code>do
</code>><code>fastq-dump  --outdir <b>outdir</b> --split-files --gzip <b>&#0036;{srr}.sra</b>;
</code>><code>rm <b>&#0036;{srr}.sra</b>
</code>><code>done</code>

>### ArrayExpress
First go to the ArrayExpress website (https://www.ebi.ac.uk/arrayexpress/) and search for the dataset with the accesion number. Click on the ENA link address under "Links", here the FASTQ FTP files (e.g. Download All) can be downloaded.

## Prepare fastq files 
Prepare the fastq files for downstream analysis. Below the processing is decribed for <i>SMART-seq</i> datasets.

#### Requiered software
<ul><li>Python3</li>
<li>Bash</li>
<li>script: Smartseq_add_cell_ID.py</li>
<li>script: merge_files.sh</li>
</ul>



>### SMART-seq/SMART-seq2
>First add a cell ID to sequence identifier line.
>SMART-seq or SMART-seq2 datasets don't have a UMI and usually each fastq file corresponds to one cell. First make the following files:
><ol><li>Generate a <b>cell ID text file</b> with two columns seperated by a comma, the first column contains the SRR name (the name of the downloaded fastq file; e.g. SRR0001) and the second column a cell ID (e.g. cell_1).</li>
><li>Generate a <b>fastq list text file</b> with a single column containing the single-end fastq file names (e.g. read 1) of all the files that need to be used.</ol>
>Run the following script:
><code>Smartseq_add_cell_ID.py --fq1 <b>fastq list text file</b> --cellid <b>cell ID text file</b> --outdir <b>outdir</b> --outprefix <b>outprefix</b> --idx <b>index sequence</b> </code>
>
>For downstream analysis merge the single cell fastq files into one fastq file per donor:<br>
><code>merge_files.sh &#92; 
<b>txt file</b> with files to merge (single column) &#92;
<b>output file name</b> e.g. merged_fastq.gz &#92;
<b>empty</b> <i>or</i> <b>-f</b> to overwrite existing file 
</code>

### Trim galore
The trim galore package is used to remove low-quality bases and sequence adapters from the reads in the processed and merged fastq file. To use Trim Galore, make sure that <i>Cutadapt</i> and <i>FastQC</i> are also installed. 

#### Requiered software
<ul><li>Python3</li>
<li>Bash</li>
<li>Trim_galore</li>
<li>Cutadapt</li>
<li>FastQC</li>
</ul>
 
><code>trim_galore --illumina &#92; 
-o <b>outdir</b> &#92;
<b>fastq file</b></code>

## Generate BAM file
When the fastq files are processed (e.g. addition of cell barcode, merged, and trimmed) the reads are mapped to a reference genome using the STAR package.  

#### Requiered software
<ul><li>Bash</li>
<li>STAR</li>
<li>wget</li>
<li>samtools</li>
<li>picard</li>
<li>java JDK (version 8!)</li>
<li>script: align_and_add_RG.sh</li>
<li>script: star_allign.sh</li>
</ul>

>### Index a genome with STAR
>First an indexed genome has to be generated, do this for each read length variety. Download a gtf file and primary assembly FASTA file from ensembl genome database, for example download by:<br> 
><code>wget <b>http://ftp.ensembl.org/pub/release-102/gtf/homo_sapiens/Homo_sapiens.GRCh38.102.gtf.gz</b>
>wget <b>http://ftp.ensembl.org/pub/release-102/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz</b></code>
>
>When making the indexed genome select as <i>sjdbOverhang</i> the read length - 1<br>
><code>STAR --runThreadN <b>integer</b> --runMode genomeGenerate --genomeDir <b>outdir</b> --genomeFastaFiles <b>genomeFastaFiles</b> --sjdbGTFfile <b>genomeGTFfile</b> --sjdbOverhang <b>ReadLength - 1</b></code>
    
>### Map to genome
>Next a script is used to map the files to previously generated indexed genome, sort the bam file based on genomic location, filtering out of multimappers, and addition of a read group ID. Make sure that the installed Java JDK version is set to 8 to support the picard package. As library tag use for example a 3 letter code, this tag is used to identify the dataset.
>
><code>align_and_add_RG.sh &#92; 
<b>fastq file</b> &#92; 
<b>library tag</b> &#92; 
<b>donor/sample ID</b> &#92; 
<b>indexed genome folder</b>
</code>


## Count genes

#### Requiered software
<ul><li>Bash</li>
<li>samtools</li>
<li>featureCounts</li>
<li>script: count_tables.sh</li>
<li>script: split_bam.sh</li>
<li>script: count_genes.sh</li>
</ul>

>### Count genes
To count genes of cells sequenced from different datasets, the pipeline counts reads only falling reads within the 3'UTR region or mitochondrial reads. In order to do this we made a GTF file that contains the longest possible 3' UTR region as well as all the mitochondrial genes (both annotated as three_prime_utr_or_mitogene). First BAM files are split in BAM files specific for each cell, using the cell ID annotation from the read ID. After this the reads are count with featureCounts with the count feature set as three_prime_utr_or_mitogene. The output file is a tsv file and the gene is annotated with the chromosome number separated by a > (e.g INS>11).

><code>count_tables.sh &#92;
<b>BAM + dir</b> &#92;
<b>GTF</b> &#92;
<b>count feature: three_prime_utr_or_mitogene or gene</b> &#92;
<b>outdir</b>
</code>




## Count matrix quality control and filtering
A script is used after generation of the gene count matrix to make plots to assess the quality of the data. This script makes the following plots and histograms: number of reads, frequency of genes, types of reads per cells, mitochondrial read fraction, GAPDH read fraction, and reads per gene. The output is used to manually set treshold criteria to filter cells and genes in the jupyter python notebook.

#### Requiered software
<ul><ul><li>Bash</li>
<li>Python 3</li>
<li>script: count_matrix_QC.py</li>
<li>notebook: T2DM-ND_dataset_generation.ipynb</li>
</ul>

>### Quality control

><code>count_matrix_QC.py &#92;
<b>TSV count matrix</b> &#92;
<b>sample ID</b> &#92;
<b>outdir</b>
</code>





## Gene expression analysis
Differential gene expression analysis and generation of plots is performed in a jupyter notebook.

#### Requiered software
<ul><li>Python 3</li>
<li>notebook: APOLgene_expression_T2DM.ipynb</li>
</ul>